# Lab 8 - Summarizing Attendance and Practice Attempts

In the last lab, you combined the information for attendance and practice quizzes into one combined table.  In this lab, you will transform these data into two summary tables.

1. For each class, you will create a table that summarizes the attendance of all students in that class.
2. For each class that has practice quizzes, you will create a table that summarized the number of attempts, overall and for each module.

## Tasks

To complete this lab, complete each of the following tasks.

In [1]:
import pandas as pd
from dfply import *

In [71]:
all_data = pd.read_csv("./attendance_and_quiz_data_combined.csv")

In [72]:
all_data.columns = ['Unnamed_0', 'Org_Defined_ID', 'UserName', 'FirstName', 'LastName',
       'Attempt_num', 'Score', 'Out_Of', 'Attempt_Start', 'Attempt_End',
       'Percent', 'Course', 'Quiz_type', 'Module']

#### Task 1 - Attendance Summary

For each of the classes contained in `attendance_example.zip`, we want to create a table with three columns: `FirstName`, `LastName`, `Attendance`.  Note that `Attendance` will represent the maximum number of attempts on the quiz, which corresponds to that students number of days they have attended.  To complete this task, do the following.

1. Write a helper function that takes a class identifier and the overall dataframe as arguments; and returns the table described above.
2. Use a loop and your helper functions to create the attendance quiz table for each class and then write the contents to a csv file.

In [73]:
course_vals = ['dsci494s7', 'stat180s18', 'stat491s1']

In [74]:
def get_attendance_table(df,identifier):
    df2 = (df 
          >> filter_by(X.Course == identifier,
                       X.Quiz_type == 'Attendance') 
          >> group_by(X.FirstName,X.LastName,X.UserName)
          >> summarize(Attendance = colmax(X.Attempt_num)) 
          >> select(X.FirstName,X.LastName,X.Attendance)      
         )
    return df2

In [75]:
for c in course_vals:
    attend_table = get_attendance_table(all_data,c)
    attend_table.to_csv('./attendance_{}.csv'.format(c), index=False)    

#### Task 2 - Practice Quiz Summary

Some of the classes contained in `attendance_example.zip` contain information about attempts on practice quizzes for four modules.  We want to create a table for each class that summarizes the practice quiz attempts.  This table should contain the following columns: `FirstName`, `LastName`, `Module 1`, `Module 2`, `Module 3`, `Module 4`, and `Total Attempts`.  Note that, for example, `Module 1` contains the total number of attempts each student made on the corresponding quiz and `Total Attempts` contains the total number of attempts on all four quizzes.


1. Create a list of class that have practice quizzes.
2. Write a helper function that takes a class identifier and the overall dataframe as arguments; and returns the table described above.
3. Use a loop and your helper functions to create the practice quiz table for each class and then write the contents to a csv file.

In [76]:
quiz_courses = ['dsci494s7', 'stat491s1']

In [81]:
def get_quiz_attempts(df,identifier):
    df2 = (df 
              >> filter_by(X.Course == identifier,
                           X.Quiz_type == 'Practice') 
              >> group_by(X.FirstName,X.LastName,X.Module)
              >> summarize(Attempts = colmax(X.Attempt_num))
              >> spread(X.Module,X.Attempts)
              >> mutate(Total_attempts = X.Module_1+X.Module_2+X.Module_3+X.Module_4)
          )
    return df2

In [82]:
for c in quiz_courses:
    attempt_table = get_quiz_attempts(all_data,c)
    attempt_table.to_csv('./quiz_attempts_{}.csv'.format(c), index=False)    